<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2017/Chapter_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 17 Data types as objects

# 17.1 Types are objects, too

In [ ]:
type(5)

<class 'int'>


In [ ]:
type(['hello', 'goodbye'])

<class 'list'>


In [ ]:
type_result = type(5)
type(type_result)

type

# 17.2 Using types

In [ ]:
type("Hello") == type("Goodbye")

True


In [ ]:
type("Hello") == type(5)

False


# 17.3 Types and user-defined classes

In [ ]:
class A:
    pass

class B(A):
    pass


In [ ]:
b = B()
type(b)

<class '__main__.B'>


In [ ]:
b.__class__

<class '__main__.B'>


In [ ]:
b_class = b.__class__
b_class == B

True


In [ ]:
b_class.__name__

'B'


In [ ]:
b_class.__bases__

(<class '__main__.A'>,)


In [ ]:
class C:
    pass

class D:
    pass

class E(D):
    pass

x = 12
c = C()
d = D()
e = E()
isinstance(x, E)

False


In [ ]:
isinstance(c, E)             #A

False


In [ ]:
isinstance(e, E)

True


In [ ]:
isinstance(e, D)             #B

True


In [ ]:
isinstance(d, E)                #C

False


In [ ]:
y = 12
isinstance(y, type(5))             #D

True


In [ ]:
issubclass(C, D)

False


In [ ]:
issubclass(E, D)

True


In [ ]:
issubclass(D, D)           #E

True


In [ ]:
issubclass(e.__class__, D)

True


# 17.5 What is a special method attribute?

### Listing 17.1 File color_module.py

In [ ]:
# Listing 17.1 File color_module.py

class Color:
    def __init__(self, red, green, blue):
        self._red = red
        self._green = green
        self._blue = blue
    def __str__(self):
        return f"Color: R={self._red:d}, G={self._green:d}, B={self._blue:d}"

In [ ]:
#from color_module import Color
c = Color(15, 35, 3)

In [ ]:
print(c)

Color: R=15, G=35, B=3


# 17.7 The `__getitem__` special method attribute

In [ ]:
class LineReader:
    def __init__(self, filename):
        self.fileobject = open(filename, 'r')              #A
    def __getitem__(self, index):
        line = self.fileobject.readline()                  #B
        if line == "":                                #C
            	self.fileobject.close()      #D
            raise IndexError         #E

        else:
            return line.split("::")[:2]                    #F


# 17.8 Giving an object full list capability

In [ ]:
class TypedList:
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)                     #A
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                          "be a list.")
        for element in initial_list:
                if not isinstance(element, self.type):
                    raise TypeError("Attempted to add an element of "
                                  "incorrect type to a typed list.")
        self.elements = initial_list[:]

In [ ]:
class TypedList:
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        self.elements = initial_list[:]
    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")
    def __setitem__(self, i, element):
        self.__check(element)
        self.elements[i] = element
    def __getitem__(self, i):
        return self.elements[i]

In [ ]:
x = TypedList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [ ]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')

## 17.9.1 Subclassing list

In [ ]:
class TypedListList(list):
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        super().__init__(initial_list)

    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")

    def __setitem__(self, i, element):
        self.__check(element)
        super().__setitem__(i, element)

In [ ]:
x = TypedListList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [ ]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')

In [ ]:
x[:]

['', '', 'Hello', 'There', '']

In [ ]:
del x[2]
x[:]

['', '', 'There', '']

In [ ]:
x.sort()
x[:]

['', '', '', 'There']

## 17.9.2 Subclassing UserList

In [ ]:
from collections import UserList
class TypedUserList(UserList):
    def __init__(self, example_element, initial_list=[]):
        self.type = type(example_element)
        if not isinstance(initial_list, list):
            raise TypeError("Second argument of TypedList must "
                            "be a list.")
        for element in initial_list:
            self.__check(element)
        super().__init__(initial_list)

    def __check(self, element):
        if type(element) != self.type:
            raise TypeError("Attempted to add an element of "
                            "incorrect type to a typed list.")
    def __setitem__(self, i, element):
        self.__check(element)
        self.data[i] = element
    def __getitem__(self, i):
        return self.data[i]

In [ ]:
x = TypedUserList("", 5 * [""])
x[2] = "Hello"
x[3] = "There"
print(x[2] + ' ' + x[3])

Hello There


In [ ]:
a, b, c, d, e = x
a, b, c, d

('', '', 'Hello', 'There')

In [ ]:
x[:]

['', '', 'Hello', 'There', '']

In [ ]:
del x[2]
x[:]

['', '', 'There', '']

In [ ]:
x.sort()
x[:]

['', '', '', 'There']

# Lab 17: Creating a string only key:value dictionary

The quick check above mentions creating a dictionary that only allows strings as keys. Let's that idea a step further and actually implement a dictionary that only allows strings for both the keys and values. This sort of dictionary might be useful for example to cache URL's and web pages in a web application.

As mentioned in discussing lists above, you would have three possible approaches - write a class from scratch, inherit from the built-in dictionary, or inherit from UserDictionary. I would suggest for the best combination of simplicity and functionality that you inherit from the built-in `dict` type and override the  `__setitem__()` method.  

In [15]:
""" Create a dictionary that allows only strings for keys and values"""

class StringDict(dict):
    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        if not isinstance(value, str):
            raise TypeError('values must be strings')
        super().__setitem__(key, value)
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        keys = any(not isinstance(_, str) for _ in self.keys())
        if keys:
            raise TypeError('keys must be strings')
        values = any(not isinstance(_, str) for _ in self.values())
        if values:
            raise TypeError('values must be strings')


In [26]:
test_dict = StringDict()
test_dict['a'] = 'b'
test_dict['c'] = 'd'
print(test_dict)


() {}
{'a': 'b', 'c': 'd'}


In [5]:
test_dict = StringDict()
test_dict['a'] = 1
test_dict[2] = 'd'
print(test_dict)

{'a': '1', '2': 'd'}


In [24]:
test_dict = StringDict([(1,2), ("a", "b")])
print(test_dict)

([(1, 2), ('a', 'b')],) {}


TypeError: keys must be strings

In [18]:
test_dict = StringDict({1:2, "a": "b"})
print(test_dict)

({'1': '2', 'a': 'b'},) {}
{'1': '2', 'a': 'b'}


In [25]:
test_dict

{'1': '2', 'a': 'b'}